In [10]:
from dqt import DBUtils, get_fut_min, get_fut_taq
import pandas as pd
from time import sleep
import numpy as np
from time import time
from datetime import timedelta



In [3]:
dbu = DBUtils()

all_varieties = ['A', 'AG', 'AL', 'AU', 'AP', 'BU', 'B', 'C', 'CF', 'CJ', 'CS', 'CU', 'CY', 'EG', 'FG', 'HC', 'I', 'J',
                'JD', 'JM', 'L', 'M', 'MA', 'NI', 'NR', 'OI', 'P', 'PB', 'PP', 'RB', 'RM', 'RU', 'SC', 'SP', 'SR', 'T',
                'TA', 'TF', 'TS', 'UR', 'V', 'Y', 'ZC', 'ZN']

all_trading_days = dbu.get_trading_day()
first_trading_day = all_trading_days[0]
print('first_trading_day: {}'.format(first_trading_day))
tick_data_first_day = '2014-01-01'

first_trading_day: 1990-12-19 00:00:00


In [11]:
#统计每个日期cs助理合约taq数据的数量
cs_main_daily_df = dbu.get_main_daily(variety='CS', start_date=tick_data_first_day)

print('cs_main_daily_df: \n', cs_main_daily_df)
num_records_df = pd.DataFrame(data={'date':cs_main_daily_df['date'], 'symbol':cs_main_daily_df['symbol']}, columns=['date', 'symbol', 'num taq records'])
print('num_records_df: \n', num_records_df)

df_size = num_records_df.shape[0]
start_time = time()
i=0
for idx, row in cs_main_daily_df.iterrows():
    i+=1
    if i%1000 == 0:
        end_time = time()
        duration = end_time - start_time
        all_duration = duration * (df_size / 1000)
        start_time = time()
        print('time approximation: {}'.format(timedelta(seconds=all_duration)))
        print('idx: ', idx)
        print('row[date]: ', row['date'])
        print('row[symbol]: ', row['symbol'])
    try:
        taq_df = get_fut_taq(row['symbol'], start_date=row['date'], end_date=row['date'])
    except:
        print('{} at {} has error get_fut_taq'.format(row['symbol'], row['date']))
        continue
    #print('taq_df: \n', taq_df)
    num_taq_records = taq_df.shape[0]
    del taq_df

    
    #print('num_records_df.loc[idx, num taq records]: ', num_records_df.loc[idx, 'num taq records'])
    num_records_df.loc[idx, 'num taq records'] = num_taq_records
    #print('num_records_df.loc[idx, num taq records]: ', num_records_df.loc[idx, 'num taq records'])
    

print('#####finished checking all days#####')
    
print('num_records_df: \n', num_records_df)

    

    





    

    

cs_main_daily_df: 
       adj_factor       date  symbol    open    high     low   close    volume  \
0       1.000000 2014-12-19  CS1505  2800.0  2834.0  2784.0  2793.0   29124.0   
1       1.000000 2014-12-22  CS1505  2795.0  2800.0  2727.0  2738.0   26170.0   
2       1.000000 2014-12-23  CS1505  2736.0  2760.0  2715.0  2740.0   18726.0   
3       1.000000 2014-12-24  CS1505  2747.0  2760.0  2741.0  2752.0    8284.0   
4       1.000000 2014-12-25  CS1505  2750.0  2750.0  2665.0  2727.0   20260.0   
5       1.000000 2014-12-26  CS1505  2727.0  2738.0  2718.0  2721.0    9158.0   
6       1.000000 2014-12-29  CS1505  2722.0  2737.0  2720.0  2731.0    5406.0   
7       1.000000 2014-12-30  CS1505  2727.0  2733.0  2710.0  2721.0    9056.0   
8       1.000000 2014-12-31  CS1505  2728.0  2728.0  2708.0  2713.0    5280.0   
9       1.000000 2015-01-05  CS1505  2718.0  2718.0  2678.0  2684.0   13408.0   
10      1.000000 2015-01-06  CS1505  2689.0  2697.0  2624.0  2630.0   28882.0   
11      

time approximation: 0:00:42.627997
idx:  999
row[date]:  2019-01-23 00:00:00
row[symbol]:  CS1905
#####finished checking all days#####
num_records_df: 
            date  symbol  num taq records
0    2014-12-19  CS1505             5849
1    2014-12-22  CS1505             7246
2    2014-12-23  CS1505             5032
3    2014-12-24  CS1505             2861
4    2014-12-25  CS1505             4932
5    2014-12-26  CS1505             2745
6    2014-12-29  CS1505             2211
7    2014-12-30  CS1505             3376
8    2014-12-31  CS1505             2444
9    2015-01-05  CS1505             3793
10   2015-01-06  CS1505             6361
11   2015-01-07  CS1505             5454
12   2015-01-08  CS1505             3676
13   2015-01-09  CS1505             3272
14   2015-01-12  CS1505             2401
15   2015-01-13  CS1505             3213
16   2015-01-14  CS1505             2494
17   2015-01-15  CS1505             2152
18   2015-01-16  CS1505             3341
19   2015-01-19  CS1505    

In [12]:
ten_pct_quantile = num_records_df['num taq records'].quantile(q=0.1)
print('ten_pct_quantile: ', ten_pct_quantile)
less_records_df = num_records_df[num_records_df['num taq records'] < ten_pct_quantile]
less_records_df = less_records_df.sort_values(by='num taq records')


ten_pct_quantile:  5168.5


In [17]:
cs_fewest_tenth_trading_days_df = num_records_df[num_records_df['num taq records'] < ten_pct_quantile].sort_values(by='num taq records')

In [19]:
cs_fewest_tenth_trading_days_df.to_csv('cs_fewest_tenth_trading_days.csv')